In [11]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from data_cleaning import *
import implementations as imp
import plots
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

COLUMN_TO_DROP = 22

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Import data

In [12]:
y_train, x_train, ids_train = load_csv_data("../data/train.csv")
y_test, x_test, ids_test = load_csv_data("../data/test.csv")

# Clean and add features

In [13]:
NUM_JETS = 4

PRI_jet_num_train = np.array([x_train[:, COLUMN_TO_DROP]]).astype(int)
print(PRI_jet_num_train.shape)
del_x_train = np.delete(x_train, COLUMN_TO_DROP, axis=1)
print(del_x_train.shape)

print()

replaced_x_train = replace_undefined_with_mean(del_x_train, UNDEFINED_VALUE)
print(replaced_x_train[0][0])
print(replaced_x_train.shape)

print()

norm_x_train, train_data_mean, train_data_std = mean_std_normalization(replaced_x_train)
print(norm_x_train[0][0])
print(norm_x_train.shape)

(1, 250000)
(250000, 29)

138.47
(250000, 29)

0.4736512480675119
(250000, 29)


In [14]:
PRI_jet_num_test = np.array([x_test[:, COLUMN_TO_DROP]]).astype(int)
print(PRI_jet_num_test[0:5])
print(PRI_jet_num_test.shape)
del_x_test = np.delete(x_test, COLUMN_TO_DROP, axis=1)
print(del_x_train.shape)

replaced_x_test = replace_undefined_with_mean(del_x_test, UNDEFINED_VALUE)
print(replaced_x_test[0][0])
print(replaced_x_test.shape)

norm_x_test, test_data_mean, test_data_std = mean_std_normalization(replaced_x_test, train_data_mean, train_data_std)
print(norm_x_test[0][0])
print(norm_x_test.shape)

[[0 1 0 ... 0 1 0]]
(1, 568238)
(250000, 29)
46.72265694612496
(568238, 29)
46.72265694612496
(568238, 29)


# The model

In [28]:
initial_w = np.zeros(norm_x_train.shape[1], dtype=np.float64)
max_iters = 100
gamma = 0.01
seed = 1
ratio = 0.5
(tr_x, tr_y, te_x,te_y) = split_data(norm_x_train, y_train, ratio, seed)
lr_loss, lr_weights = imp.reg_logistic_regression(tr_y, tr_x, 0.1, initial_w, max_iters, gamma)
lr_loss

-89706859.29232088

In [29]:
y_validation = predict_labels(lr_weights, te_x)
score = sum(y_validation == te_y)/len(te_y)
score

0.73224

In [ ]:
gamma_list = [0.001,0.01,0.1] #np.logspace(-7, -1, 10)
degree_list = [10, 12] #range(7, 15)
max_iters = 100
ratio = 0.5
seed = 1
b_size = 1

best_gamma = np.zeros(4)
best_degree = np.zeros(4, int)
best_score = np.zeros(4)

for degree in degree_list:
    for i in range(NUM_JETS):
        curr_x = norm_x_train[PRI_jet_num_train[0,:]==i]
        curr_y = y_train[PRI_jet_num_train[0,:]==i]
        
        (tr_x, tr_y, te_x,te_y) = split_data(curr_x, curr_y, ratio, seed)
        
        px_tr = build_poly(tr_x, degree)
        px_te = build_poly(te_x, degree)
        
        for gamma in gamma_list:
            initial_w = np.zeros(px_tr.shape[1], dtype=np.float64)
            _, w = imp.logistic_regression(tr_y, px_tr, initial_w, max_iters, gamma)
            y_validation = predict_labels(w, px_te)
            score = sum(y_validation == te_y)
            
            if score > best_score[i]:
                best_gamma[i] = gamma
                best_score[i] = score
                best_degree[i] = degree
    print(degree)

10


## Hva kan vi cross validate?
- Degrees, hvor mange degrees ska vi legge til i den polynomiske utvidelsen av dataen vår?
- Lambda, for ridge regression og og den andre logistic regressionene
- Gamma, learning rate

In [ ]:
for i in range(NUM_JETS):
    curr_x = norm_x_train[PRI_jet_num_train[0,:]==i]
    curr_y = y_train[PRI_jet_num_train[0,:]==i]
    
    px_tr = build_poly(curr_x,best_degree[i])
    _, w = imp.ridge_regression(curr_y,px_tr, best_gamma[i])
    
    curr_x_test = norm_x_test[PRI_jet_num_test[0,:]==i]
    px_test = build_poly(curr_x_test,best_degree[i])

    y_test[PRI_jet_num_test[0,:]==i] = predict_labels(w, px_test)

In [ ]:
sum(best_score)/len(y_train)

In [27]:
best_score

array([14439.,  9979.,  6079.,  2994.])